In this homework, we'll deploy the credit scoring model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.

In [1]:
! docker build -t zoomcamp-model:hw10 .

[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                 

In [2]:
# ! docker run -it --rm -p 9696:9696 --name credit_container zoomcamp-model:hw10

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'get_credit': True, 'get_credit_probability': <value>}
```

Here `<value>` is the probability of getting a credit card.

In [3]:
! python q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}


## Question 2

What's the version of `kind` that you have? 

In [4]:
! kind --version

kind version 0.20.0


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```


## Question 3

Now let's test if everything works. Use `kubectl` to get the list of running services. 

What's `CLUSTER-IP` of the service that is already running there? 

In [5]:
! kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.27.3) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [6]:
! kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:42317
CoreDNS is running at https://127.0.0.1:42317/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [7]:
! kubectl get services --all-namespaces -o=wide

NAMESPACE     NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)                  AGE    SELECTOR
default       kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP                  101s   <none>
kube-system   kube-dns     ClusterIP   10.96.0.10   <none>        53/UDP,53/TCP,9153/TCP   93s    k8s-app=kube-dns


## Question 4

To be able to use the docker image we previously created (`zoomcamp-model:hw10`),
we need to register it with `kind`.


In [8]:
! kind load docker-image zoomcamp-model:hw10

Image: "zoomcamp-model:hw10" with ID "sha256:1f91e9996624ad0132c9522f07e705787049cd9b6cfbdbf057f9053463d10c5a" not yet present on node "kind-control-plane", loading...


## Question 5

Now let's create a deployment config. 

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

In [9]:
! kubectl apply -f deployment.yaml

deployment.apps/credit created


In [10]:
! kubectl get deployment -o wide

NAME     READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                SELECTOR
credit   0/1     0            0           2s    credit       zoomcamp-model:hw10   app=credit


In [11]:
! kubectl get pods -o wide

No resources found in default namespace.


## Question 6

Let's create a service for this deployment.

In [12]:
! kubectl apply -f service.yaml

service/credit created


In [13]:
! docker stop credit_container

credit_container


In [14]:
# ! kubectl port-forward service/credit 9696:80

In [15]:
! python q6_test.py

{'get_credit': True, 'get_credit_probability': 0.726936946355423}
